Prophet Paper - https://peerj.com/preprints/3190/

In [ ]:
#Instala Prophet
#!pip install fbprophet

In [ ]:
#Carga paquetes generales
import numpy as np
import pandas as pd
from datetime import datetime
import itertools

# visual libraries
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import plotly.express as px
import os
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('ggplot')
from seaborn import kdeplot

# pruebas estadisticas de estacionalidad y causalidad
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
f


In [ ]:
#Carga Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Define la ruta raiz del proyecto
ruta="/content/drive/MyDrive/AIre/BD/"

In [ ]:
#Carga datos de una estación
estac='MER'
df=pd.read_csv(ruta + estac +'2_2015_2020.csv')
df.date2=pd.to_datetime(df['date2'],format="%Y-%m-%d %H:%M:%S")
df=df.set_index('date2')
df.drop(columns=['WDR'],inplace=True,axis=1)      #en este modelo no se va a analizar WDR, para incluirla se requiere hacer la conversión de WSP-WDR a X-Y (de coordenadas polares a cartesianas)
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41107 entries, 2015-01-01 00:00:00 to 2019-09-30 00:00:00
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        41107 non-null  object 
 1   Date        41107 non-null  object 
 2   Hour        41107 non-null  int64  
 3   year        41107 non-null  int64  
 4   id_station  41107 non-null  object 
 5   CO          35929 non-null  float64
 6   NO          36387 non-null  float64
 7   NO2         36387 non-null  float64
 8   NOX         36387 non-null  float64
 9   O3          38239 non-null  float64
 10  PA          36465 non-null  float64
 11  PBa         8491 non-null   float64
 12  PM10        34288 non-null  float64
 13  PM2.5       34288 non-null  float64
 14  PMCO        34288 non-null  float64
 15  RH          39759 non-null  float64
 16  SO2         37428 non-null  float64
 17  TMP         37702 non-null  float64
 18  WSP         38729 non-null  flo

# Evalua si la series son estacionarias y presentan causalidad

In [ ]:
# Evaluación de estacionariedad
# Se requiere que las series no tengan datos faltantes
# Considerando una base sin faltantes
# para lo cual se relleno previamente con los perfiles horarios




#carga la base con relleno de faltantes
df_all= pd.read_csv(ruta+'Sitios/' + estac +'_rell00.csv')
df_all.date2=pd.to_datetime(df['date2'],format="%Y-%m-%d %H:%M:%S")
df_all=df.set_index('date2')

#Correlación
df_corr=df_all[['Hour', 'Month', 'year', 'id_station', 'CO',
       'NO', 'NO2', 'NOX', 'O3', 'PA', 'PM10', 'PM2.5', 'PMCO', 'RH', 'SO2',
       'TMP', 'WDR', 'WSP']].corr(method='spearman')

#listado de variables con correlación mayor o igual que 0.25
var_corr=df_corr[df_corr['PM2.5'].abs() >= 0.25]['PM2.5'].sort_values(ascending=False).index

#listado de variables con correlación menor que 0.25
var_no_corr=df_all.columns[~df_all.columns.isin(var_corr)][7:]


#Función para evaluar estacionalidad
#Null Hypothesis - Series possesses a unit root and hence is not stationary
#Alternate Hypothesis - Series is stationary

def adfuller_test(variable):
  result = adfuller(df[variable])
  print(f'Test Statistics: {result[0]}')
  print(f'p-value: {result[1]}')
  print(f'Critial Values: {result[4]}')
  print(variable)
  if result[1] > 0.05 :
    print('Series is not Stationary')
  else:
    print('Series is Stationary')
    print('')

#Función para evaluar causalidad entre X y Y
# H0: Xt does not granger causes Yt
# H1: Xt granger causes Yt  

def grangercausality_tests(df, variable1, variable2, max_lags):
  results=grangercausalitytests(df[[variable1,variable2]], max_lags, verbose=False)
  p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
  print('p values por lag entre {} y {}'.format(variable1,variable2))
  print(p_values)
  print('')



#Evaluar estacionalidad en variables con mayor correlación
for i in var_corr:
  adfuller_test(i)

#Evaluar causalidad en variables con mayor correlación
for j in var_corr[1:]:
  grangercausality_tests(df_all, 'PM2.5', j, 24)
for j in var_corr[1:]:
  grangercausality_tests(df_all, j , 'PM2.5', 24)


#Evaluar estacionalidad en variables con menor correlación
for i in var_no_corr:
  adfuller_test(i)

#Evaluar causalidad en variables con menor correlación  
for j in var_no_corr:
  grangercausality_tests(df_all, 'PM2.5', j, 24)
for j in var_no_corr:
  grangercausality_tests(df_all, j, 'PM2.5', 24)      